### Source  www.agoda.com (Task 1 HOTEL DATA)

#### <span style="color:blue;">Im sorry but, I cant guarantee this code will work on your machine as I dont have the resources to test it on another environment.</span>
    
<span style="color:Red;">Please note:</span> 
<span style="color:green;">This code will require FIREWALL PERMISSION ON FIRST RUN</span> as it accesses the internet.</span><span style="color:blue"><strong>You have the option to accept or decline these settings based on your preference, I respect and value your security concern.</strong></span>
<span style="color:Red;">If you decided to decline, just feel free to check the displayed output below each code block</span></span><span style="color:blue"><strong> or reach out to me so I can show you a LIVE DEMO :), Thank you.</strong></span>

#### Note: Not all asked data from the task was extracted as the other info are not available on the page of the website. I opt to not abused the webscrapper to avoid my IP address getting banned.

Mark:
ok - with data,
x - no data

1. Name of the hotel - ok
2. Location - ok 
3. Amneties - ok
4. Price range - ok, but it is the exact price not the range
5. No. of available rooms - ok
6. Other data - ok, (review score and overall review)
7. Reviews - ok

#### <span style="color:blue;"> The instruction says nothing against combining/using BeautifulSoup with other scrapper to increase its performance so I choose Selenium to partnered with it in order to extract data

### Please install the following (run the installation below  <span style="color:red;">in order</span>) - to ensure no error in running this notebook

In [ ]:
pip install --upgrade pip

In [ ]:
pip install webdriver-manager

In [ ]:
pip install msedge-selenium-tools

In [ ]:
pip install beautifulsoup4

In [ ]:
pip install pandas

In [ ]:
pip install -U selenium

####  After installing the packages, <span style="color:blue;"> please restart and clear the kernel by running the code below</span> and then proceed to import the libraries

In [ ]:
# Run this code to restart the kernel and apply the newly installed libraries.
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

### Imported libraries

In [3]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
import random
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.edge.service import Service
from bs4 import BeautifulSoup
import platform
from msedge.selenium_tools import Edge, EdgeOptions

### To check OS if you are in Windows or Mac
##### <span style="color:blue;"> Please don't forget to run this code block and dont worry about this as it will not install anything on your laptop/PC as I already included in the submission folder the necessary driver

In [4]:
def chks_plat():
    if platform.system() == 'Windows':
        # Set the WebDriver executable path for Windows
        service  ='msedgedriver.exe'
    else:
        # Set the WebDriver executable path for macOS (this is not yet tested)
        service ='msedgedriver'
        
    return service

In [5]:
# run this code to assign driver
CHKS = chks_plat()
# as Im running this code on windows Ive got the webdriver with .exe file extension
CHKS

'msedgedriver.exe'

## EXTRACTION
### Scrapping hotel details 
#### <span style="color:blue;"> The function below will get all the Hotel names, Location, Amneties, Price and Other data from the given URL 

In [6]:
# will get the Hotel name, Location, Amneties, Price, and other data
def scrape(url_x): 
    service = Service(executable_path=CHKS)
    options = webdriver.EdgeOptions()
    options.add_argument("--headless")
    driver = webdriver.Edge(service=service, options=options)

    
    # Use Selenium to open the webpage
    driver.get(url_x)

    # Number of times to scroll down
    num_scrolls = 2

    # Delay between each scroll action (in seconds)
    scroll_delay = random.randint(5,6)  # Adjust as needed
    
    time.sleep(random.randint(3,4))
    
        # To bypass pop up

    try:
        bypass_button = driver.find_element(By.XPATH, '//*[@id="pricefreeze-promo-popup"]/div[2]/div/div/div/div[3]/button')
        bypass_button.click()
    except NoSuchElementException:
        pass 
    
    time.sleep(random.randint(3,4))
    
    # To bypass pop up calendar
    try:
        # Check if the specific div element exists
        if driver.find_element(By.CLASS_NAME, 'Popup WideRangePicker WideRangePicker--checkIn'):
            try:
                bypass_button = driver.find_element(By.XPATH, '//*[@id="check-in-box"]/div')
                bypass_button.click()
            except NoSuchElementException:
                pass
    except NoSuchElementException:
        pass

    time.sleep(random.randint(2,3))
    
    # Perform scrolling
    for i in range(num_scrolls):
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)
        time.sleep(scroll_delay)

    # Retrieve the entire page source
    page_source = driver.page_source

    # Parse the page source with BeautifulSoup
    soup = BeautifulSoup(page_source, 'html.parser')

    # Find the hotel element by tag, class, and attribute
    tag_lst = ['div', # price
                'p', # hotel name
               'span', # location
               'p', # overall_score
               'p', # overall_rate
               'p', # Amneties
                'div'] # rooms
    class_lst = ['Box-sc-kv6pi1-0 hRUYUu StickyNavPrice__priceDetail--lowerText StickyNavPrice__priceDetail--defaultColor', # price
             'HeaderCerebrum__Name', # hotel name
             'Spanstyled__SpanStyled-sc-16tp9kb-0 gwICfd kite-js-Span HeaderCerebrum__Address', # location
             'sc-jrAGrp sc-kEjbxe fzPhrN gOeEsn', # overall_score
             'sc-jrAGrp sc-kEjbxe bvAJVt gOeEsn', # overall_rate
             'Typographystyled__TypographyStyled-sc-j18mtu-0 keaLUr kite-js-Typography', # Amneties
             'RoomGrid-titleCountersText'] # rooms
    
    x_list = []
    
    # Evaluate pulled elements
    for tag, clss in zip(tag_lst, class_lst):
        elements = soup.find_all(tag, class_=clss)
        if tag == 'div' and clss == 'Box-sc-kv6pi1-0 hRUYUu StickyNavPrice__priceDetail--lowerText StickyNavPrice__priceDetail--defaultColor' and len(elements) != 0:
            elements = soup.find(tag, class_=clss)
            # removing all spaces and joing the peso sign with value.
            x_list.extend([' '.join([element for element in elements.text.split() if element.strip()])])
        elif len(elements) != 0:
            elements = soup.find_all(tag, class_=clss)
            x_list.extend([element.text.strip() for element in elements])
        else:
            # Append an empty string
            x_list.append('')
 
    # Close the browser
    driver.quit()
    
    return x_list

### <span style="color:blue;"> The first batch of hotel URLs to be scrapped. (10 links)

In [7]:
x_list = ['https://www.agoda.com/belmont-hotel-mactan/hotel/cebu-ph.html?finalPriceView=1&isShowMobileAppPrice=false&cid=-42&numberOfBedrooms=&familyMode=false&adults=1&children=0&rooms=1&maxRooms=0&checkIn=2023-09-23&isCalendarCallout=false&childAges=&numberOfGuest=0&missingChildAges=false&travellerType=-1&showReviewSubmissionEntry=false&currencyCode=PHP&isFreeOccSearch=false&isCityHaveAsq=false&tspTypes=16&los=1&searchrequestid=a84c77c0-af0c-418e-b3ee-1554904f591f',
          'https://www.agoda.com/waterfront-cebu-city-hotel-and-casino/hotel/cebu-ph.html?finalPriceView=1&isShowMobileAppPrice=false&cid=-42&numberOfBedrooms=&familyMode=false&adults=1&children=0&rooms=1&maxRooms=0&checkIn=2023-09-23&isCalendarCallout=false&childAges=&numberOfGuest=0&missingChildAges=false&travellerType=-1&showReviewSubmissionEntry=false&currencyCode=PHP&isFreeOccSearch=false&isCityHaveAsq=false&tspTypes=7,6,16&los=1&searchrequestid=24676d74-52bf-4211-aa22-1837e1b25b6e',
          'https://www.agoda.com/quest-hotel-and-conference-center-cebu_5/hotel/cebu-ph.html?finalPriceView=1&isShowMobileAppPrice=false&cid=-42&numberOfBedrooms=&familyMode=false&adults=1&children=0&rooms=1&maxRooms=0&checkIn=2023-09-23&isCalendarCallout=false&childAges=&numberOfGuest=0&missingChildAges=false&travellerType=-1&showReviewSubmissionEntry=false&currencyCode=PHP&isFreeOccSearch=false&isCityHaveAsq=false&tspTypes=7,6,16&los=1&searchrequestid=c3c884af-1419-4c71-a24b-ecf14379fbf3',
          'https://www.agoda.com/cebu-quincentennial-hotel/hotel/cebu-ph.html?finalPriceView=1&isShowMobileAppPrice=false&cid=-42&numberOfBedrooms=&familyMode=false&adults=1&children=0&rooms=1&maxRooms=0&checkIn=2023-09-23&isCalendarCallout=false&childAges=&numberOfGuest=0&missingChildAges=false&travellerType=-1&showReviewSubmissionEntry=false&currencyCode=PHP&isFreeOccSearch=false&isCityHaveAsq=false&los=1&searchrequestid=bcc280cc-21cd-44a4-908a-cf47f25457de',
          'https://www.agoda.com/radisson-blu-cebu/hotel/cebu-ph.html?finalPriceView=1&isShowMobileAppPrice=false&cid=-42&numberOfBedrooms=&familyMode=false&adults=1&children=0&rooms=1&maxRooms=0&checkIn=2023-09-23&isCalendarCallout=false&childAges=&numberOfGuest=0&missingChildAges=false&travellerType=-1&showReviewSubmissionEntry=false&currencyCode=PHP&isFreeOccSearch=false&isCityHaveAsq=false&tspTypes=15,6,16&los=1&searchrequestid=d0fd0f7b-d620-4061-8a20-c7bece5ad5dc',
          'https://www.agoda.com/savoy-hotel-mactan-newtown/hotel/cebu-ph.html?finalPriceView=1&isShowMobileAppPrice=false&cid=-42&numberOfBedrooms=&familyMode=false&adults=1&children=0&rooms=1&maxRooms=0&checkIn=2023-09-23&isCalendarCallout=false&childAges=&numberOfGuest=0&missingChildAges=false&travellerType=-1&showReviewSubmissionEntry=false&currencyCode=PHP&isFreeOccSearch=false&isCityHaveAsq=false&tspTypes=7,6,16&los=1&searchrequestid=ef793eeb-e328-41a8-ad76-9820ffcc88f8',
          'https://www.agoda.com/seda-ayala-center-cebu/hotel/cebu-ph.html?finalPriceView=1&isShowMobileAppPrice=false&cid=-42&numberOfBedrooms=&familyMode=false&adults=1&children=0&rooms=1&maxRooms=0&checkIn=2023-09-23&isCalendarCallout=false&childAges=&numberOfGuest=0&missingChildAges=false&travellerType=-1&showReviewSubmissionEntry=false&currencyCode=PHP&isFreeOccSearch=false&isCityHaveAsq=false&tspTypes=7,6,16&los=1&searchrequestid=1ead381a-0204-47aa-a302-860b1fc0e750',
          'https://www.agoda.com/fili-urban-resort/hotel/cebu-ph.html?finalPriceView=1&isShowMobileAppPrice=false&cid=-42&numberOfBedrooms=&familyMode=false&adults=1&children=0&rooms=1&maxRooms=0&checkIn=2023-09-23&isCalendarCallout=false&childAges=&numberOfGuest=0&missingChildAges=false&travellerType=-1&showReviewSubmissionEntry=false&currencyCode=PHP&isFreeOccSearch=false&isCityHaveAsq=false&tspTypes=15,6,16,-1&los=1&searchrequestid=aded0ae6-c964-42ad-8217-cfc9193412f2',
          'https://www.agoda.com/bai-hotel/hotel/cebu-ph.html?finalPriceView=1&isShowMobileAppPrice=false&cid=-42&numberOfBedrooms=&familyMode=false&adults=1&children=0&rooms=1&maxRooms=0&checkIn=2023-09-23&isCalendarCallout=false&childAges=&numberOfGuest=0&missingChildAges=false&travellerType=-1&showReviewSubmissionEntry=false&currencyCode=PHP&isFreeOccSearch=false&isCityHaveAsq=false&tspTypes=15,6,16&los=1&searchrequestid=f2f43f3f-3dc6-4d05-9e18-2bda78719004',
          'https://www.agoda.com/one-central-hotel/hotel/cebu-ph.html?finalPriceView=1&isShowMobileAppPrice=false&cid=-42&numberOfBedrooms=&familyMode=false&adults=1&children=0&rooms=1&maxRooms=0&checkIn=2023-09-23&isCalendarCallout=false&childAges=&numberOfGuest=0&missingChildAges=false&travellerType=-1&showReviewSubmissionEntry=false&currencyCode=PHP&isFreeOccSearch=false&isCityHaveAsq=false&tspTypes=7,6,16&los=1&searchrequestid=3d7b53b9-5aff-4a86-ac6e-ae7f16d3efa3'
         ]

#### <span style="color:blue;">Start scrapping

In [8]:
# Batch 1 scrapped
hotel_name_list = []

# This will loop thru list of links and supply each of them the scrape function
for i in range(0, len(x_list)):
    print(f'URL No. {i+1} out of {len(x_list)}, Please wait . . .')
    hotel_name_list.append(scrape(x_list[i]))
    time.sleep(random.randint(4,6))
    if i+1 == 10:
        print('Done')

URL No. 1 out of 10, Please wait . . .
URL No. 2 out of 10, Please wait . . .
URL No. 3 out of 10, Please wait . . .
URL No. 4 out of 10, Please wait . . .
URL No. 5 out of 10, Please wait . . .
URL No. 6 out of 10, Please wait . . .
URL No. 7 out of 10, Please wait . . .
URL No. 8 out of 10, Please wait . . .
URL No. 9 out of 10, Please wait . . .
URL No. 10 out of 10, Please wait . . .
Done


In [10]:
# result from batch 1 scrapped data
for i in hotel_name_list:
    print(f'{i}\n')

['from ₱ 2,703', 'Belmont Hotel Mactan', 'Newtown Boulevard, Mactan Belmont Luxury Hotel, Mactan Island, Cebu, Philippines, 6016', '8.8', 'Excellent', 'Great for activities', 'Sparkling clean', 'Excellent room comfort & quality', 'Front desk [24-hour]', 'Free Wi-Fi in all rooms!', 'Valet parking', 'Free Wi-Fi in all rooms!', 'Luggage storage', 'Breakfast [free]', '1 room type with only 3 total room offers still available']

['from ₱ 3,367', 'Waterfront Cebu City Hotel and Casino', '1 Salinas Drive Lahug, Cebu City, Cebu, Philippines, 6000', '8.2', 'Excellent', 'Hygiene Plus', '720 meters to Ayala Center', 'Check-in [24-hour]', 'Airport transfer', 'Excellent breakfast', 'Airport transfer', 'Valet parking', 'Shuttle service', 'Free Wi-Fi in all rooms!', 'Fitness center', '3 room types | 8 room offers']

['from ₱ 2,414', 'Quest Hotel and Conference Center Cebu', 'Archbishop Reyes Avenue, Cebu City, Cebu, Philippines, 6000', '8.6', 'Excellent', 'Located in heart of Cebu', 'Check-in [24-hou

### <span style="color:blue;"> The second batch of hotel URLs to be scrapped. (10 links)

In [11]:
# the second 10 URLs to be scrapped. (batch 2)
x_list2 = ['https://www.agoda.com/hop-inn-hotel-cebu-city/hotel/cebu-ph.html?finalPriceView=1&isShowMobileAppPrice=false&cid=-42&numberOfBedrooms=&familyMode=false&adults=1&children=0&rooms=1&maxRooms=0&checkIn=2023-09-23&isCalendarCallout=false&childAges=&numberOfGuest=0&missingChildAges=false&travellerType=-1&showReviewSubmissionEntry=false&currencyCode=PHP&isFreeOccSearch=false&isCityHaveAsq=false&tspTypes=7,6,16&los=1&searchrequestid=1069b7b8-eb74-47ff-890a-e85995a9b128',
           'https://www.agoda.com/mandarin-plaza-hotel/hotel/cebu-ph.html?finalPriceView=1&isShowMobileAppPrice=false&cid=-42&numberOfBedrooms=&familyMode=false&adults=1&children=0&rooms=1&maxRooms=0&checkIn=2023-09-23&isCalendarCallout=false&childAges=&numberOfGuest=0&missingChildAges=false&travellerType=-1&showReviewSubmissionEntry=false&currencyCode=PHP&isFreeOccSearch=false&isCityHaveAsq=false&tspTypes=7,6,16&los=1&searchrequestid=d3978b4d-577c-4301-8f6b-900542bc162c',
           'https://www.agoda.com/bayfront-hotel-cebu-capitol-site/hotel/cebu-ph.html?finalPriceView=1&isShowMobileAppPrice=false&cid=-42&numberOfBedrooms=&familyMode=false&adults=1&children=0&rooms=1&maxRooms=0&checkIn=2023-09-23&isCalendarCallout=false&childAges=&numberOfGuest=0&missingChildAges=false&travellerType=-1&showReviewSubmissionEntry=false&currencyCode=PHP&isFreeOccSearch=false&isCityHaveAsq=false&tspTypes=6,16&los=1&searchrequestid=2ce2a826-aaef-4fb7-b557-da4587ef50f7',
           'https://www.agoda.com/secret-paradise-moalboal/hotel/all/cebu-ph.html?finalPriceView=1&isShowMobileAppPrice=false&cid=-42&numberOfBedrooms=&familyMode=false&adults=1&children=0&rooms=1&maxRooms=0&checkIn=2023-09-23&isCalendarCallout=false&childAges=&numberOfGuest=0&missingChildAges=false&travellerType=-1&showReviewSubmissionEntry=false&currencyCode=PHP&isFreeOccSearch=false&isCityHaveAsq=false&los=1&searchrequestid=4746dde6-3fd6-4f05-82ee-a14fd28710ef',
           'https://www.agoda.com/hotel-elizabeth-cebu/hotel/cebu-ph.html?finalPriceView=1&isShowMobileAppPrice=false&cid=-42&numberOfBedrooms=&familyMode=false&adults=1&children=0&rooms=1&maxRooms=0&checkIn=2023-09-23&isCalendarCallout=false&childAges=&numberOfGuest=0&missingChildAges=false&travellerType=-1&showReviewSubmissionEntry=false&currencyCode=PHP&isFreeOccSearch=false&isCityHaveAsq=false&tspTypes=7,6,16&los=1&searchrequestid=759092fa-fa74-4202-b6b5-c314d303bcbe',
           'https://www.agoda.com/seda-central-bloc-cebu/hotel/cebu-ph.html?finalPriceView=1&isShowMobileAppPrice=false&cid=-42&numberOfBedrooms=&familyMode=false&adults=1&children=0&rooms=1&maxRooms=0&checkIn=2023-09-23&isCalendarCallout=false&childAges=&numberOfGuest=0&missingChildAges=false&travellerType=-1&showReviewSubmissionEntry=false&currencyCode=PHP&isFreeOccSearch=false&isCityHaveAsq=false&tspTypes=7,6,16&los=1&searchrequestid=01b52b39-a6e3-4356-97d9-905c84700ff9',
           'https://www.agoda.com/cebu-parklane-international-hotel/hotel/cebu-ph.html?finalPriceView=1&isShowMobileAppPrice=false&cid=-42&numberOfBedrooms=&familyMode=false&adults=1&children=0&rooms=1&maxRooms=0&checkIn=2023-09-23&isCalendarCallout=false&childAges=&numberOfGuest=0&missingChildAges=false&travellerType=-1&showReviewSubmissionEntry=false&currencyCode=PHP&isFreeOccSearch=false&isCityHaveAsq=false&tspTypes=7,6,16&los=1&searchrequestid=463c2905-9d0f-4eb1-aff3-fa9ae16d1df2',
           'https://www.agoda.com/summit-galleria-cebu_2/hotel/cebu-ph.html?finalPriceView=1&isShowMobileAppPrice=false&cid=-42&numberOfBedrooms=&familyMode=false&adults=1&children=0&rooms=1&maxRooms=0&checkIn=2023-09-23&isCalendarCallout=false&childAges=&numberOfGuest=0&missingChildAges=false&travellerType=-1&showReviewSubmissionEntry=false&currencyCode=PHP&isFreeOccSearch=false&isCityHaveAsq=false&tspTypes=6,16&los=1&searchrequestid=3ef92f8b-948b-4ab9-a291-0bf73fd490d9',
           'https://www.agoda.com/kiwi-hotel_2/hotel/cebu-ph.html?finalPriceView=1&isShowMobileAppPrice=false&cid=-42&numberOfBedrooms=&familyMode=false&adults=1&children=0&rooms=1&maxRooms=0&checkIn=2023-09-23&isCalendarCallout=false&childAges=&numberOfGuest=0&missingChildAges=false&travellerType=-1&showReviewSubmissionEntry=false&currencyCode=PHP&isFreeOccSearch=false&isCityHaveAsq=false&tspTypes=7,3,-1&los=1&searchrequestid=c48e45a2-5062-40cc-83ef-98cd86ea6174',
           'https://www.agoda.com/the-noble-cebu/hotel/cebu-ph.html?finalPriceView=1&isShowMobileAppPrice=false&cid=-42&numberOfBedrooms=&familyMode=false&adults=1&children=0&rooms=1&maxRooms=0&checkIn=2023-09-23&isCalendarCallout=false&childAges=&numberOfGuest=0&missingChildAges=false&travellerType=-1&showReviewSubmissionEntry=false&currencyCode=PHP&isFreeOccSearch=false&isCityHaveAsq=false&tspTypes=6&los=1&searchrequestid=6205e614-97dd-4c6a-9e0f-db38e1c4ef5b'
          ]

#### <span style="color:blue;">Start scrapping

In [12]:
# Batch 2 scraped
hotel_name_list2 = []
                                                                                                                        
for i in range(0, len(x_list2)):
    print(f'URL No. {i+1} out of {len(x_list2)}, Please wait . . .')
    hotel_name_list2.append(scrape(x_list2[i]) )
    time.sleep(random.randint(4,6))
    if i+1 == 10:
        print('Done')

URL No. 1 out of 10, Please wait . . .
URL No. 2 out of 10, Please wait . . .
URL No. 3 out of 10, Please wait . . .
URL No. 4 out of 10, Please wait . . .
URL No. 5 out of 10, Please wait . . .
URL No. 6 out of 10, Please wait . . .
URL No. 7 out of 10, Please wait . . .
URL No. 8 out of 10, Please wait . . .
URL No. 9 out of 10, Please wait . . .
URL No. 10 out of 10, Please wait . . .
Done


In [13]:
# result from batch 2 scrapped data
for i in hotel_name_list2:
    print(f'{i}\n')

['from ₱ 1,800', 'Hop Inn Hotel Cebu City', 'Lot 5 & 6 Block 20, Samar Loop, Cebu Business Park, Cebu City, Cebu, Philippines, 6000', '9.0', 'Exceptional', 'Hygiene Plus', 'Sparkling clean', 'Inside city center', 'Top Value', 'Check-in [24-hour]', 'Car park', 'Free Wi-Fi in all rooms!', 'Luggage storage', 'Laundry service', 'Room service', '2 room types | only 4 total room offers still available']

['from ₱ 3,357', 'Mandarin Plaza Hotel', 'Archbishop Reyes Corner Escario Street, Cebu City, Cebu, Philippines, 6000', '8.3', 'Excellent', 'Located in heart of Cebu', 'Great food & dining', 'Great Swimming pool', 'Great View', 'Hygiene Plus', 'Car park', 'Free Wi-Fi in all rooms!', 'Front desk [24-hour]', 'Luggage storage', 'Contactless check-in/out', '2 room types | only 2 total room offers still available']

['from ₱ 1,999', 'Bayfront Hotel Cebu - Capitol Site', '60d M. P. Yap St, Cebu City, Cebu, Philippines, 6000', '8.7', 'Excellent', 'Located in heart of Cebu', 'Great for activities', '

### <span style="color:blue;"> The final batch of hotel URLs to be scrapped. (5 links)

In [14]:
# the last 5 URLs to be scrapped. (batch 3)
x_list3 = ['https://www.agoda.com/bayfront-hotel-cebu/hotel/cebu-ph.html?finalPriceView=1&isShowMobileAppPrice=false&cid=-42&numberOfBedrooms=&familyMode=false&adults=1&children=0&rooms=1&maxRooms=0&checkIn=2023-09-23&isCalendarCallout=false&childAges=&numberOfGuest=0&missingChildAges=false&travellerType=-1&showReviewSubmissionEntry=false&currencyCode=PHP&isFreeOccSearch=false&isCityHaveAsq=false&tspTypes=7,6,16&los=1&searchrequestid=0b2af572-5ed0-4ed5-bce5-76262813b789',
           'https://www.agoda.com/castle-peak-hotel/hotel/cebu-ph.html?finalPriceView=1&isShowMobileAppPrice=false&cid=-42&numberOfBedrooms=&familyMode=false&adults=1&children=0&rooms=1&maxRooms=0&checkIn=2023-09-23&isCalendarCallout=false&childAges=&numberOfGuest=0&missingChildAges=false&travellerType=-1&showReviewSubmissionEntry=false&currencyCode=PHP&isFreeOccSearch=false&isCityHaveAsq=false&tspTypes=6,16&los=1&searchrequestid=30921617-b30d-4f57-96a6-3e2f950e4159',
           'https://www.agoda.com/holiday-inn-cebu-city/hotel/cebu-ph.html?finalPriceView=1&isShowMobileAppPrice=false&cid=-42&numberOfBedrooms=&familyMode=false&adults=1&children=0&rooms=1&maxRooms=0&checkIn=2023-09-23&isCalendarCallout=false&childAges=&numberOfGuest=0&missingChildAges=false&travellerType=-1&showReviewSubmissionEntry=false&currencyCode=PHP&isFreeOccSearch=false&isCityHaveAsq=false&tspTypes=7,6,16&los=1&searchrequestid=91e973e6-ccee-43bd-8cc3-8d24392be8b4',
           'https://www.agoda.com/main-hotel-and-suites/hotel/cebu-ph.html?finalPriceView=1&isShowMobileAppPrice=false&cid=-42&numberOfBedrooms=&familyMode=false&adults=1&children=0&rooms=1&maxRooms=0&checkIn=2023-09-23&isCalendarCallout=false&childAges=&numberOfGuest=0&missingChildAges=false&travellerType=-1&showReviewSubmissionEntry=false&currencyCode=PHP&isFreeOccSearch=false&isCityHaveAsq=false&tspTypes=6,-1&los=1&searchrequestid=d60d855f-53ca-4320-a9c4-ee5cfe58cbd9',
           'https://www.agoda.com/primeway-suites-cebu/hotel/cebu-ph.html?finalPriceView=1&isShowMobileAppPrice=false&cid=-42&numberOfBedrooms=&familyMode=false&adults=1&children=0&rooms=1&maxRooms=0&checkIn=2023-09-23&isCalendarCallout=false&childAges=&numberOfGuest=0&missingChildAges=false&travellerType=-1&showReviewSubmissionEntry=false&currencyCode=PHP&isFreeOccSearch=false&isCityHaveAsq=false&tspTypes=6,16&los=1&searchrequestid=7b46b5cd-85f1-4f2c-8c56-2290dc3872a8'
          ]

#### <span style="color:blue;">Start scrapping

In [15]:
# Batch 3 scraped
hotel_name_list3 = []
                                                                                                                        
for i in range(0, len(x_list3)):
    print(f'URL No. {i+1} out of {len(x_list3)}, Please wait . . .')
    hotel_name_list3.append(scrape(x_list3[i]) )
    time.sleep(random.randint(4,6))
    if i+1 == 5:
        print('Done')

URL No. 1 out of 5, Please wait . . .
URL No. 2 out of 5, Please wait . . .
URL No. 3 out of 5, Please wait . . .
URL No. 4 out of 5, Please wait . . .
URL No. 5 out of 5, Please wait . . .
Done


In [17]:
# result from batch 3 scrapped data
for i in hotel_name_list3:
    print(f'{i}\n')

['', 'Bayfront Hotel Cebu - North Reclamation', 'Kaohsiung Street, North Reclamation Area, Cebu City, Cebu, Philippines, 6000', '8.4', 'Excellent', '840 meters to public transportation', 'Great food & dining', 'Great for activities', 'Great View', 'Hygiene Plus', 'Airport transfer', 'Car park', 'Shuttle service', 'Free Wi-Fi in all rooms!', '']

['from ₱ 1,422', 'Castle Peak Hotel', 'F.Cabahug Corner, President Quezon St., Villa Aurora, Mabolo, Cebu City, Philippines, Cebu City, Cebu, Philippines, 06000', '8.0', 'Excellent', 'Check-in [24-hour]', 'Airport transfer', 'Excellent breakfast', 'Excellent view', 'Excellent food & dining', 'Airport transfer', 'Valet parking', 'Free Wi-Fi in all rooms!', '2 room types | 6 room offers']

['from ₱ 4,500', 'Holiday Inn Cebu City', 'Lot 5 & 6 Block 20, Samar Loop, Cebu Business Park, Cebu City, Cebu, Philippines, 6000', '9.1', 'Exceptional', 'Hygiene Plus', 'Sparkling clean', '440 meters to Ayala Center', 'Top Value', 'Check-in [24-hour]', 'Free W

### <span style="color:blue;"> Combing the 3 batch of scrapped hotel data

In [18]:
# Combining the 3 batch  of scrapped data
hotel_list = hotel_name_list + hotel_name_list2 + hotel_name_list3
hotel_list

[['from ₱ 2,703',
  'Belmont Hotel Mactan',
  'Newtown Boulevard, Mactan Belmont Luxury Hotel, Mactan Island, Cebu, Philippines, 6016',
  '8.8',
  'Excellent',
  'Great for activities',
  'Sparkling clean',
  'Excellent room comfort & quality',
  'Front desk [24-hour]',
  'Free Wi-Fi in all rooms!',
  'Valet parking',
  'Free Wi-Fi in all rooms!',
  'Luggage storage',
  'Breakfast [free]',
  '1 room type with only 3 total room offers still available'],
 ['from ₱ 3,367',
  'Waterfront Cebu City Hotel and Casino',
  '1 Salinas Drive Lahug, Cebu City, Cebu, Philippines, 6000',
  '8.2',
  'Excellent',
  'Hygiene Plus',
  '720 meters to Ayala Center',
  'Check-in [24-hour]',
  'Airport transfer',
  'Excellent breakfast',
  'Airport transfer',
  'Valet parking',
  'Shuttle service',
  'Free Wi-Fi in all rooms!',
  'Fitness center',
  '3 room types | 8 room offers'],
 ['from ₱ 2,414',
  'Quest Hotel and Conference Center Cebu',
  'Archbishop Reyes Avenue, Cebu City, Cebu, Philippines, 6000',



### Scrapping hotel reviews
#### <span style="color:blue;"> This function will scrapped 15 reviews from the hotels we previously get data from

In [42]:
# to get the reviews on the hotel and the name of person who gave the review
def scrape_rev(url_x):
    service = Service(executable_path=CHKS)
    options = webdriver.EdgeOptions()
    options.add_argument("--headless")
    driver = webdriver.Edge(service=service, options=options)

    # Use Selenium to open the webpage
    driver.get(url_x)

    # Number of times to scroll down
    num_scrolls = random.randint(8,9)

    # Delay between each scroll action (in seconds)
    scroll_delay = random.randint(2,3)  # Adjust as needed
    
    time.sleep(random.randint(2,3))
    
        # To bypass pop up

    try:
        bypass_button = driver.find_element(By.XPATH, '//*[@id="pricefreeze-promo-popup"]/div[2]/div/div/div/div[3]/button')
        bypass_button.click()
    except NoSuchElementException:
        pass 
    
    time.sleep(random.randint(2,3))
    
    # To bypass pop up calendar
    try:
        # Check if the specific div element exists
        if driver.find_element(By.CLASS_NAME, 'Popup WideRangePicker WideRangePicker--checkIn'):
            try:
                bypass_button = driver.find_element(By.XPATH, '//*[@id="check-in-box"]/div')
                bypass_button.click()
            except NoSuchElementException:
                pass
    except NoSuchElementException:
        pass
 

    # Perform scrolling
    for i in range(num_scrolls):
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)
        time.sleep(scroll_delay)

    # Retrieve the entire page source
    page_source = driver.page_source

    # Parse the page source with BeautifulSoup
    soup = BeautifulSoup(page_source, 'html.parser')
    
    # Find the hotel name element by class and attribute
    rev_element = soup.find_all(['p', 'div'], class_=['Review-comment-bodyText',
                                                       'Review-comment-reviewer'])

     
    rev_list = [element.text.strip() for element in rev_element]
 
    # Close the browser
    driver.quit()
    
    list_rev = []
    
    for x in rev_list:
        list_rev.append(x)  
    
    return list_rev       

### <span style="color:blue;"> The first batch of hotel review URLs to be scrapped. (10 links)

In [20]:
# Batch 1 scrapped review
rev_list = []
                                                                                                                        
for i in range(0, len(x_list)):
    print(f'URL No. {i+1} out of {len(x_list)}, Please wait . . .')
    rev_list.append(scrape_rev(x_list[i])) # using the same list of URL from batch 1 (x_list)
    time.sleep(random.randint(4,6))
    if i+1 == 10:
        print('Done')

URL No. 1 out of 10, Please wait . . .
URL No. 2 out of 10, Please wait . . .
URL No. 3 out of 10, Please wait . . .
URL No. 4 out of 10, Please wait . . .
URL No. 5 out of 10, Please wait . . .
URL No. 6 out of 10, Please wait . . .
URL No. 7 out of 10, Please wait . . .
URL No. 8 out of 10, Please wait . . .
URL No. 9 out of 10, Please wait . . .
URL No. 10 out of 10, Please wait . . .
Done


In [21]:
# Checking scrapped review for batch 1
rev_list

[['David from Australia',
  'Couple',
  'Double Room',
  'Stayed 7 nights in September 2023',
  'Very professionally organised and run  hotel',
  'Janina from Philippines',
  'Couple',
  'Double Room',
  'Stayed 2 nights in September 2023',
  'Location is great with many restaurants and shopping outlets near the area. The Lapu-Lapu shrine is also at walking distance. Staff are very welcoming and approachable. The place and room is clean. We were able to enjoy the pool, gym, and dry sauna.',
  'Mary from Philippines',
  'Couple',
  'Double Room',
  'Stayed 1 night in August 2023',
  "Kinda slow process on the check in and check out. Pool is small, sauna and gym is okay. I was so disappointed with the breakfast buffet. I think of all the budget hotels I've been to in Cebu, this is ranked as my lowest",
  'Casline from Singapore',
  'Business traveler',
  'Double Room',
  'Stayed 4 nights in September 2023',
  'Everything was ok for this new hotel except the air conditioning in the room w

### <span style="color:blue;"> The second batch of hotel review URLs to be scrapped. (10 links)

In [35]:
# Batch 2 scrapped review
rev_list2 = []
                                                                                                                        
for i in range(0, len(x_list2)):
    print(f'URL No. {i+1} out of {len(x_list2)}, Please wait . . .')
    rev_list2.append(scrape_rev(x_list2[i])) # using the same list of URL from batch 2 (x_list2)
    time.sleep(random.randint(4,6))
    if i+1 == 10:
        print('Done')

URL No. 1 out of 10, Please wait . . .
URL No. 2 out of 10, Please wait . . .
URL No. 3 out of 10, Please wait . . .
URL No. 4 out of 10, Please wait . . .
URL No. 5 out of 10, Please wait . . .
URL No. 6 out of 10, Please wait . . .
URL No. 7 out of 10, Please wait . . .
URL No. 8 out of 10, Please wait . . .
URL No. 9 out of 10, Please wait . . .
URL No. 10 out of 10, Please wait . . .
Done


In [36]:
# Checking scrapped review for batch 2
rev_list2

[['DIVINA from Philippines',
  'Business traveler',
  'Standard Double Room',
  'Stayed 3 nights in September 2022',
  "Location - right at the center of Cebu, and very close to Ayala Center.\n\nRooms - just the right size but very clean, good AC,  clean bathroom with super clean towels, nice desk for quick office works and good internet connection. Also, sockets near the bed and in work area are available.\n\nSleep - nice bed and pillows! And I love the subtle fresh laundry scent.\n\nParking - you can rent a car for fast client coverage. They have good parking space in upper floors with elevator directly covering lobby to different levels. No problem from rain or heat of the sun - you'll smell fresh from your room to your clients!\n\nService - I love their staff from receptionist, guards to room maintenance. All smile, super fast check-in and check out . My officemate got fast response for her room connection concern.\n\nYou'll get two free bottles of mineral water everyday, refills o

### <span style="color:blue;"> The last batch of hotel review URLs to be scrapped. (5 links)

In [28]:
# Batch 3 scrapped review
rev_list3 = []

for i in range(0, len(x_list3)):
    print(f'URL No. {i+1} out of {len(x_list3)}, Please wait . . .')
    rev_list3.append(scrape_rev(x_list3[i])) # using the same list of URL from batch 3 (x_list3)
    time.sleep(random.randint(4,6))
    if i+1 == 5:
        print('Done')

URL No. 1 out of 5, Please wait . . .
URL No. 2 out of 5, Please wait . . .
URL No. 3 out of 5, Please wait . . .
URL No. 4 out of 5, Please wait . . .
URL No. 5 out of 5, Please wait . . .
Done


In [29]:
# Checking Scrape list for batch 3
rev_list3

[['Mikaela from Philippines',
  'Solo traveler',
  'Superior Room',
  'Stayed 3 nights in January 2023',
  'Travelled to  Cebu with my mom for 4 days & 3 nights.  Room was so clean and spacious also the bathroom didnt expect it to be that big for 2 pax ☺️\n\nBed was big and clean but the comforter was really heavy haha no biggies but it really keeps u warm to the point na lalakasan mo na aircon 🤣 floor and walls are clean no random insects haha \n\nBathroom and cabinets were so spacious; heater was not that hot warm lang..kettle was working tho yung ref medyo madumi :)\n\nFacilities like gym and elev were great! Very accessible and location in general super near the malls! \n\nBuffet breakfast was 9/10 some dishes are okay not that tasty..in general daming varieties so all good padin 😊 worth it\n\nRoom price super worth it with bfast and considering the room for 2 pax hehe\n\nHotel staff were very hospitable as well, we left our baggages on our last day to roam around pa and it was ver

### <span style="color:blue;"> Combining the 3 batch of scrapped hotel reviews data

In [41]:
# Combining the 3 batch of scrapped data reviews
hotel_rev = rev_list + rev_list2 + rev_list3
hotel_rev

[['David from Australia',
  'Couple',
  'Double Room',
  'Stayed 7 nights in September 2023',
  'Very professionally organised and run  hotel',
  'Janina from Philippines',
  'Couple',
  'Double Room',
  'Stayed 2 nights in September 2023',
  'Location is great with many restaurants and shopping outlets near the area. The Lapu-Lapu shrine is also at walking distance. Staff are very welcoming and approachable. The place and room is clean. We were able to enjoy the pool, gym, and dry sauna.',
  'Mary from Philippines',
  'Couple',
  'Double Room',
  'Stayed 1 night in August 2023',
  "Kinda slow process on the check in and check out. Pool is small, sauna and gym is okay. I was so disappointed with the breakfast buffet. I think of all the budget hotels I've been to in Cebu, this is ranked as my lowest",
  'Casline from Singapore',
  'Business traveler',
  'Double Room',
  'Stayed 4 nights in September 2023',
  'Everything was ok for this new hotel except the air conditioning in the room w

## TRANSFORMATION
### For hotel details
### <span style="color:blue;"> Now were gonna assign each data to their respective list

In [31]:
# Just creating a copy of the list as backup
import copy

hotel_copy = copy.deepcopy(hotel_list)

In [32]:
# Extracting hotel names and storing it into a list
hotel_names = []
for lst in hotel_copy:
    hotel_names.append(lst[1])
    
# Extracting hotel names and storing it into a list    
location = []
for lst in hotel_copy:
    location.append(lst[2])   
    
# Extracting Amneties and storing it into a list
amne = []
for lst in hotel_copy:
    amne.append(lst[5:-1])
    
# Extracting price range and storing it into a list
price = []
for lst in hotel_copy:
    price.append(lst[0])
    
# Extracting number of available rooms and storing it into a list
rooms = []
for lst in hotel_copy:
    rooms.append(lst[-1])
    
# Extracting other data (overall score, overall ratings) and storing it into a list
Other_data = []
for lst in hotel_copy:
    Other_data.append((lst[3], lst[4]))

print(f'{hotel_names},\n\n {location}, \n\n {amne}, \n\n {price},\n\n {rooms}, \n\n {Other_data}')

['Belmont Hotel Mactan', 'Waterfront Cebu City Hotel and Casino', 'Quest Hotel and Conference Center Cebu', 'Cebu Quincentennial Hotel', 'Radisson Blu Cebu', 'Savoy Hotel Mactan Newtown', 'Seda Ayala Center Cebu', 'Fili Hotel at Nustar Cebu', 'bai Hotel', 'One Central Hotel', 'Hop Inn Hotel Cebu City', 'Mandarin Plaza Hotel', 'Bayfront Hotel Cebu - Capitol Site', 'Secret paradise moalboal', 'Hotel Elizabeth Cebu', 'Seda Central Bloc Cebu', 'Cebu Parklane International Hotel', 'Summit Galleria Cebu (Multiple Use Hotel)', 'Kiwi Hotel', 'The Noble Cebu', 'Bayfront Hotel Cebu - North Reclamation', 'Castle Peak Hotel', 'Holiday Inn Cebu City', 'Main Hotel and Suites', 'Primeway Suites Cebu'],

 ['Newtown Boulevard, Mactan Belmont Luxury Hotel, Mactan Island, Cebu, Philippines, 6016', '1 Salinas Drive Lahug, Cebu City, Cebu, Philippines, 6000', 'Archbishop Reyes Avenue, Cebu City, Cebu, Philippines, 6000', '23 Minore Park, Cardinal Rosales Avenue, Cor. Pope John Paul II Ave (formerly, Juan L

## LOADING
### Hotel details and reviews

In [43]:
# First is we import pandas then store the list of data into data dictionary
import pandas as pd

data = {'Hotel_name': hotel_names, 'Location': location, 'Amneties': amne, 'Price': price, 'Available_rooms': rooms, 'Other_data': Other_data, 'Reviews': hotel_rev}

In [44]:
# we store all the data into 1 dataframe
df = pd.DataFrame(data)
df

# Note: during scrapping, blank price and available rooms means that the hotel is fully book as 
# the webpage hides the html tag which holds the data. 

,Hotel_name,Location,Amneties,Price,Available_rooms,Other_data,Reviews
0,Belmont Hotel Mactan,"Newtown Boulevard, Mactan Belmont Luxury Hotel...","[Great for activities, Sparkling clean, Excell...","from ₱ 2,703",1 room type with only 3 total room offers stil...,"(8.8, Excellent)","[David from Australia, Couple, Double Room, St..."
1,Waterfront Cebu City Hotel and Casino,"1 Salinas Drive Lahug, Cebu City, Cebu, Philip...","[Hygiene Plus, 720 meters to Ayala Center, Che...","from ₱ 3,367",3 room types | 8 room offers,"(8.2, Excellent)","[Shane from Canada, Couple, Ambassador Room, S..."
2,Quest Hotel and Conference Center Cebu,"Archbishop Reyes Avenue, Cebu City, Cebu, Phil...","[Located in heart of Cebu, Check-in [24-hour],...","from ₱ 2,414",2 room types | 9 room offers,"(8.6, Excellent)","[Jezza from Philippines, Family with teens, De..."
3,Cebu Quincentennial Hotel,"23 Minore Park, Cardinal Rosales Avenue, Cor. ...","[470 meters to Ayala Center, Rated highly by S...",,,"(8.9, Excellent)","[Michael from United States, Couple, Deluxe Qu..."
4,Radisson Blu Cebu,Serging Osmena Boulevard Corner Juan Luna Aven...,"[100 meters to Sirao Flower Garden, Great food...","from ₱ 7,290",3 room types | 15 room offers,"(8.9, Excellent)","[Vincent from Philippines, Couple, Superior Ro..."
5,Savoy Hotel Mactan Newtown,"The Mactan Newtown, Mactan Newtown Boulevard, ...","[Great food & dining, Great for activities, Gr...","from ₱ 2,275",4 room types | 14 room offers,"(8.2, Excellent)","[Roland from Philippines, Group, Superior Delu..."
6,Seda Ayala Center Cebu,"Cardinal Rosales Ave.Cebu Business Park, Cebu ...","[Inside city center, Rated highly by Solo trav...","from ₱ 6,361",2 room types | only 4 total room offers still ...,"(8.8, Excellent)","[Steve from Singapore, Family with young child..."
7,Fili Hotel at Nustar Cebu,"Kawit Island South Road Properties , Cebu City...","[Hygiene Plus, Sparkling clean, 990 meters to ...",,,"(8.6, Excellent)","[Renald from United States, Couple, Deluxe Roo..."
8,bai Hotel,"Ouano Ave Mandaue, Mandaue City, Cebu, Philipp...","[Rated highly by Solo travelers, Great food & ...","from ₱ 3,683",4 room types | 11 room offers,"(8.9, Excellent)","[David from United Kingdom, Solo traveler, Del..."
9,One Central Hotel,"Corner Leon Kilat and Sanciangko Streets, Cebu...","[240 meters to public transportation, Great fo...","from ₱ 1,911",1 room type with only 2 total room offers stil...,"(8.6, Excellent)","[Darianne from Philippines, Couple, Twin Delux..."


In [45]:
# Saving it to csv file
df.to_csv('Hotel.csv', index = False)